In [1]:
import pyro
import pyro.optim as optim
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from torch.distributions import constraints
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing

In [2]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)
pyro.clear_param_store()

In [3]:
# loading datasets
df = pd.read_csv('wine.data', header=None)
X = df.iloc[:,1:]
y = df.iloc[:,0].astype(int)
y = y - 1

In [4]:
def model(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(naive_bayes.current_class_probs))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(naive_bayes.current_class_probs), constraint=constraints.positive)
    params = pyro.distributions.Normal(loc=mu_param, scale=sigma_param).to_event(1)
    with pyro.plate("map", len(X)):
        pyro.sample("probs", params, obs=X)

def guide(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(naive_bayes.current_class_probs))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(naive_bayes.current_class_probs), constraint=constraints.positive)
    probs_prior = pyro.distributions.Normal(loc=mu_param, scale=sigma_param).to_event(1)
    return pyro.sample("probs", probs_prior, infer={'is_auxiliary': True})

def train(X):
    pyro.clear_param_store()
    num_iterations=5000
    optim = pyro.optim.Adam({"lr": 0.03})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(X))
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(X)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

In [5]:
class NaiveBayesClassifier:
    def __init__(self, use_svi=True):
        self.use_svi = use_svi
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        
        self.available_y = np.unique(y)
        self.num_features = X.shape[1]
        self._count_y_prob()
        self.params_for_probs = list()
        
        for target in self.available_y:
            self.X_current_class = torch.from_numpy(X[y==target])
            if (self.use_svi):
                self.current_class_probs = torch.from_numpy(np.random.randn(self.num_features))
                train(self.X_current_class)
                mu = pyro.param("mu")
                sigma = pyro.param("sigma")
                sigma2 = sigma*sigma # pyro uses std instead of var
            self.params_for_probs.append(torch.stack([mu, sigma2], dim=0))
            
        for i in range(len(self.params_for_probs)):
            self.params_for_probs[i] = self.params_for_probs[i].detach().numpy()
            
    def _count_y_prob(self):
        total_quantity = len(self.y)
        self.p_y = [np.count_nonzero(self.y == i) / total_quantity for i in self.available_y]
    
    def predict(self, X):
        predicted = list()
        for i in range(len(X)):
            predicted.append(self._predict_one_example(i, X[i, :]))
        return np.asarray(predicted)

    def _predict_one_example(self, i: int, x: np.ndarray):
        certainity_for_ys = list()
        for y in self.available_y:  # for every class
            certainity_for_ys.append(self.p_y[y])
            for i in range(len(x)):  # for every feature
                certainity_for_ys[-1] *= self._p_xi_on_condition_y(i, y, x[i])
        return self.available_y[certainity_for_ys.index(max(certainity_for_ys))]
    
    def _p_xi_on_condition_y(self, feature_index, y_index, x_i):
#         print(y_index)
#         print(feature_index)
#         print(self.params_for_probs)
        multiplier = 1 / np.sqrt(2 * np.pi * self.params_for_probs[y_index][1, feature_index])
        exp = - (x_i - self.params_for_probs[y_index][0, feature_index]) ** 2 / (2 * self.params_for_probs[y_index][1, feature_index])
        return multiplier * np.power(np.e, exp)

In [6]:
naive_bayes = NaiveBayesClassifier()
# naive_bayes.fit(X, y)

In [7]:
def crossval_research(data, target):
    splitter = StratifiedKFold(n_splits=7, shuffle=True)
    split_set_generator = splitter.split(data, target)

    # trainning and testing
    y_pred = list()
    y_true = list()

    for train_indices, test_indices in split_set_generator:
        X_train = data[train_indices]
        Y_train = target[train_indices]
        naive_bayes.fit(X_train, Y_train)
        y_pred.extend(naive_bayes.predict(data[test_indices]))
        y_true.extend(target[test_indices])

    confusion = metrics.confusion_matrix(y_true, y_pred)
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred, average=None)
    recall = metrics.recall_score(y_true, y_pred, average=None)
    f1_score = metrics.f1_score(y_true, y_pred, average=None)

    return {"confusion": confusion, "accuracy": accuracy, "precision": precision,
            "recall": recall, "f1_score": f1_score}

In [8]:
class_metrics = crossval_research(X.values, y.values)

In [9]:
param_store = pyro.get_param_store()
for key in param_store.keys():
    print(key)
    
print(pyro.param("mu"))
print(pyro.param("sigma"))

mu
sigma
tensor([13.1893,  3.4074,  2.4454, 21.4881,  5.1305,  1.6817,  0.7857,  0.4471,
         1.1824,  7.4214,  0.6876,  1.6893,  4.7425], dtype=torch.float64,
       requires_grad=True)
tensor([  0.5369,   1.1036,   0.1830,   2.3555,  81.9035,   0.3581,   0.2985,
          0.1276,   0.4134,   2.3700,   0.1118,   0.2808, 111.4774],
       dtype=torch.float64, grad_fn=<AddBackward0>)


In [10]:
print(class_metrics)

{'confusion': array([[58,  1,  0],
       [ 3, 66,  2],
       [ 0,  0, 48]]), 'accuracy': 0.9662921348314607, 'precision': array([0.95081967, 0.98507463, 0.96      ]), 'recall': array([0.98305085, 0.92957746, 1.        ]), 'f1_score': array([0.96666667, 0.95652174, 0.97959184])}


### Reasearch of model and guide

In [ ]:
X_class_0 = X[y==0].values
print(X_class_0.mean(0))
print(X_class_0.std(0))

means = torch.zeros_like(torch.from_numpy(X_class_0[0,:]))
stds = torch.ones_like(torch.from_numpy(X_class_0[0,:]))

X_class_0 = torch.from_numpy(X_class_0)

In [ ]:
def model(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(means))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(stds), constraint=constraints.positive)
    params = pyro.distributions.Normal(loc=mu_param, scale=sigma_param).to_event(1)
    with pyro.plate("map", len(X)):
        pyro.sample("probs", params, obs=X)

def guide(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(means))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(stds), constraint=constraints.positive)
    probs_prior = pyro.distributions.Normal(loc=mu_param, scale=sigma_param).to_event(1)
    return pyro.sample("probs", probs_prior, infer={'is_auxiliary': True})

def train(X):
    pyro.clear_param_store()
    num_iterations=10000
    optim = pyro.optim.Adam({"lr": 0.01})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(X))
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(X)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

In [ ]:
svi, losses = train(X_class_0)

In [ ]:
pyro_mu = pyro.param("mu")
pyro_sigma = pyro.param("sigma")
data_mu = X_class_0.mean(0)
data_sigma = X_class_0.std(0)

In [ ]:
print(data_mu)
print(pyro_mu)
print(pyro_mu - data_mu)
print()
print(data_sigma)
print(pyro_sigma)
print(pyro_sigma - data_sigma)